# Importanción de librerías 

In [3]:
import tensorflow as tf
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.externals import joblib as joblib
import numpy as np
import librosa
import os

In [4]:
tf.test.is_gpu_available()

True

# Generación de ficheros

Esta función se encarga de generar un archivo txt de manera que este contenga 2 elementos por fila ('name_file','etiqueta') de esta manera permitirá poder entrenar la red.

In [5]:
import numpy as np
import os
def generate_file_data(dir,name):
    directory=dir
    #el nombre de los archivos posee el primer dígito en el nombre de esta forma permitirá etiquetarlos.
    a={'0':'cero','1':'uno','2':'dos','3':'tres','4':'cuatro','5':'cinco','6':'seis','7':'siete','8':'ocho','9':'nueve'}
    da=os.listdir(directory)
    # ordena los archivos
    da.sort()
    file = open(dir+name+'.txt',"w")
    for filename in da:
        if '.wav' in filename:
            file.write(filename+','+a[filename[0]]+'\n')
    file.close() 
    # genera el fichero
    with open(directory+'/'+name+'.txt') as f:
        read_data = f.read()
        f.closed
    read_data=read_data.split('\n')
    read_data=read_data[0:len(read_data)-1]
    return read_data

# Encoding words with One Hot Encoding
En esta sección se usará one hot encoding para representar las palabras

In [6]:
onehot=joblib.load('OneHotEncoding/onehot')

### Se muestran las funciones para codificar y decodificar

In [7]:
def encode(x):# tomará un array de string y lo transformada a encode
    return onehot.transform(x.reshape(-1,1)).toarray()
def decode(x):
    return onehot.inverse_transform(x)

# MFCC 


In [8]:
def mfcc_features(DIR,list_dir):
    mfcc_audios=[]
    for dir in list_dir:
        wave, sr = librosa.load(DIR+dir, mono=True)
        features= librosa.feature.mfcc(wave, sr,n_mfcc=13)
        try:
            features=np.pad(features,((0,0),(0,150-len(features[0]))),mode='constant', constant_values=0)
        except OSError as err:
            print(dir)
        mfcc_audios.append(features)
    mfcc_audios=np.array(mfcc_audios)
    return mfcc_audios

In [9]:
def prepare_data(dir,name):
    file = open(dir+name)
    f=file.read()
    file.close()
    f=f.split('\n')
    f=f[0:len(f)-1]
    labels=[]
    names_audios=[]
    for i in f:
        j=i.split(',')
        names_audios.append(j[0])
        labels.append(j[1])
    labels=np.array(labels)
    onehot= encode(labels)
    mfcc=mfcc_features(dir,names_audios)
    print(name+' OK')
    return mfcc,onehot

En esta sección se definen los distintos modelos a aplicar en este seminario

# Load Model 

En esta sección se cargará el modelo propuesto para realizar las pruebas

In [10]:
import tensorflow as tf
from keras.utils import plot_model

Using TensorFlow backend.


In [11]:
def load_model(dir):
    m=tf.keras.models.load_model(dir)
    return m

In [12]:
model=load_model('Models/prueba/LSTM950_3')

# Recording audios and predict them

In [13]:
import ipywidgets as widgets
from IPython.display import display
import pyaudio
import wave
import sklearn.externals.joblib as joblib

In [14]:
def prepare(name='prueba'):
    generate_file_data('data/pruebas/',name)
    data=prepare_data('data/pruebas/',name+'.txt')
    return data

In [53]:
def test_prec():
    data=prepare()
    testx, testy = data[0],data[1]
    testx=np.matrix.transpose(testx,[0,2,1])
    print(model.evaluate(testx,testy))

In [38]:
def predict_number(dir):
    onda, sr = librosa.load(dir, mono=True)
    features= librosa.feature.mfcc(onda, sr,n_mfcc=13)
    features=np.pad(features,((0,0),(0,150-len(features[0]))),mode='constant', constant_values=0)
    f=np.matrix.transpose(np.array([features]),[0,2,1])
    label=model.predict_classes(f)
    texto=decode(np.eye(10)[label])
    print(texto)

In [54]:
test_prec()

prueba.txt OK
13/13 [==============================] - 0s 32ms/step
[1.6168429851531982, 0.6153846383094788]


In [39]:
predict_number('data/pruebas/9_prueba_b.wav')

[['nueve']]


In [40]:
predict_number('data/pruebas/6_prueba_b.wav')


[['seis']]


In [42]:
predict_number('data/pruebas/1_v_v1.wav')

[['dos']]


In [44]:
predict_number('data/pruebas/1_v_v2.wav')

[['cero']]


In [45]:
predict_number('data/pruebas/1_v_v3.wav')


[['cero']]


In [46]:
predict_number('data/pruebas/2_vi_v1.wav')

[['dos']]


In [47]:
predict_number('data/pruebas/2_vi_v2.wav')

[['dos']]


In [48]:
predict_number('data/pruebas/2_vi_v3.wav')

[['dos']]


In [49]:
predict_number('data/pruebas/3_v_v1.wav')

[['tres']]


In [50]:
predict_number('data/pruebas/3_v_v2.wav')

[['tres']]


In [51]:
predict_number('data/pruebas/3_v_v3.wav')

[['tres']]
